In [5]:
library(stringr)
library(RColorBrewer)
library(parallel)
library(pheatmap)

In [35]:
setwd("../analysis/")

In [6]:
dir = '/nfs/lab/projects/islet_cytok/analysis/deseq_ATAC_NEW/'

In [37]:
fi = list.files(dir)

In [38]:
li= list()
for (ff in fi ){
    
    file = read.table(paste0(dir, ff), header=T, row.names = 1)
    
    nm = paste( str_split_fixed(ff, "\\.",2)[1], "down", sep=".")
     li[[nm]] <- na.omit(rownames(file)[file$padj<0.1 & file$log2FoldChange>0])## the orientation was swapped

    nm = paste( str_split_fixed(ff, "\\.",2)[1], "up", sep=".")
    li[[nm]] <- na.omit(rownames(file)[file$padj<0.1 & file$log2FoldChange<0])## the orientation was swapped
}

In [39]:
mm = matrix(lapply(li, length), ncol = 2, byrow = T)[c(1:4,6,5,7),1:2]
colnames(mm)= c("Down", "Up") 

In [40]:
##union =c(length(unique(unlist(li[c(1,3,5,7)]))) , length(unique(unlist(li[c(2,4,6,8)])))) why only the 4?
union =c(length(unique(unlist(li[c(1,3,5,7,9,11,13)]))) , length(unique(unlist(li[c(2,4,6,8,10,12,14)]))))

In [41]:
mm= rbind(union, mm)

In [42]:
pdf("cytokine_figures/Summary_DACs_cytok.pdf")
par(mfrow=c(2,2),mar = c(4,4,4,0))
b = barplot(t(mm), las=2, legend.text = colnames(mm), ylim=c(0,50000), col = brewer.pal(9, "PuOr")[c(3,7)] )
text(x = b, y =mm[,1], labels = paste0(round(unlist(mm[,1])/1000,1),"k") , pos = 1)
text(x = b, y = as.numeric(mm[,2])+ as.numeric(mm[,1]), labels = paste0(round(unlist(mm[,2])/1000,1),"k") , pos = 1)
dev.off()

png 
  2

In [43]:
u =  data.frame(unique(unlist(li)))
for(n in 1:length(li)){
    u[,n+1] =   u[,1] %in% li[[n]]
}


In [44]:
rownames(u)=u[,1]
u=u[,-1]

In [45]:
colnames(u) = names(li)

In [46]:
colSums(u)

cytokine_24hr_Hi.down    cytokine_24hr_Hi.up cytokine_24hr_Hi2.down 
                 16821                  18309                   3514 
  cytokine_24hr_Hi2.up  cytokine_24hr_Lo.down    cytokine_24hr_Lo.up 
                  8884                      1                     25 
cytokine_24hr_Lo2.down   cytokine_24hr_Lo2.up  cytokine_48hr_Hi.down 
                     1                     19                   7112 
   cytokine_48hr_Hi.up   cytokine_6hr_Hi.down     cytokine_6hr_Hi.up 
                  9336                   5751                  10003 
 cytokine_72hr_Hi.down    cytokine_72hr_Hi.up 
                  9037                  12684

In [47]:
bed_full = cbind(str_split_fixed(rownames(u), "\\-",4)[,2:4], rownames(u))

In [48]:
write.table(bed_full, "deseq_ATAC_NEW_bedfiles/All_differential.bed", sep="\t", quote=F, row.names=F)

In [67]:
for (cc in colnames(u)){
    bed = subset(bed_full, u[,cc]==TRUE)
    #write.table(bed, paste0("deseq_ATAC_NEW_bedfiles/", cc, ".bed"), sep="\t", quote=F, row.names=F, col.names=F)
    write.table(bed[,1:3], paste0("/nfs/lab/projects/islet_cytok/analysis/ldsc/", cc, ".bed"), sep="\t", quote=F, row.names=F, col.names=F)
}

In [52]:
dim(bed_full)

[1] 44969     4

In [54]:
dim(file)

[1] 165884      6

In [47]:
write.table( subset(bed_full, rowSums(u[,c(2,4,6,8,10,12,14)])>0),"deseq_ATAC_NEW_bedfiles/All_differential_up.bed", sep="\t", quote=F, row.names=F, col.names=F)
write.table( subset(bed_full, rowSums(u[,c(1,3,5,7,9,11,13)])>0),"deseq_ATAC_NEW_bedfiles/All_differential_down.bed", sep="\t", quote=F, row.names=F, col.names=F)

In [68]:
write.table( subset(bed_full[,1:3], rowSums(u[,c(2,4,6,8,10,12,14)])>0),"/nfs/lab/projects/islet_cytok/analysis/ldsc/All_differential_up.bed", sep="\t", quote=F, row.names=F, col.names=F)
write.table( subset(bed_full[,1:3], rowSums(u[,c(1,3,5,7,9,11,13)])>0),"/nfs/lab/projects/islet_cytok/analysis/ldsc/All_differential_down.bed", sep="\t", quote=F, row.names=F, col.names=F)

In [48]:
length(unique(rownames(u)))

[1] 44969

In [66]:
write.table(u, "deseq_ATAC_NEW_bedfiles/annotated_peaks.tsv", sep="\t", quote=F)

In [ ]:
#annotatePeaks.pl TNFa_24h_hi.bed hg19  -m known41.motif -noann > TNFa_24h_hi.NFKB_1.txt

In [ ]:
#findMotifsGenome.pl TNFa_24h_hi.bed hg19 out -find known41.motif > TNFa_24h_hi.NFKB_v2.txt

In [ ]:
#findMotifsGenome.pl 24h_hi.bed hg19 out -find known41.motif > 24h_hi.bed_nfkb.txt

In [ ]:
#annotatePeaks.pl 24h_hi.bed hg19 -m known41.motif -noann > 24h_hi.bed_nfkb_v1.txt

### Plot Motif enrichemnt

In [186]:
homer_dir = '/nfs/lab/elisha/islets/cytokine/HOMER/homer_dec15/'
#list.files(homer_dir)
files2 = c('union_up','union_down', 'hiVhi2_up','hiVhi2_down','hi2_24_up','hi2_24_down')

In [11]:
#pdf("cytokine_figures/Plot_motifs.pdf", height = 6, width=12)
pdf("cytokine_figures/Plot_motifs_2.pdf")

#par(mfcol=c(2,4), mar = c(4,4,4,0))
par(mfcol=c(2,3), mar = c(4,4,4,0))
for (exp in files2){
hom= read.table( paste0(homer_dir, exp, '/knownResults.txt'), header=F, fill=T, check.names=F, skip=1)
hom$fold_change = as.numeric(gsub('%',"", hom$V7)) /(as.numeric(gsub('%',"", hom$V9))+0.1) ##pseudocount for inf (only V2)
hom$Motif = toupper(str_split_fixed(hom$V1, "\\(",5)[,1])
hom[,"-logP"] = -(hom$V4)

hom=hom[order(hom$fold_change, decreasing=T),]
hom=hom[order(hom[,'-logP'], decreasing=T),]
    
plot(hom[,c('fold_change', '-logP')], pch = 19, 
     col = (hom$V5<0.05)+1  )
if(sum(hom$V5<0.05)>0){
    text (hom[1:25,c('fold_change', '-logP')], hom$Motif[1:20], 
      pos=2, cex=0.7,offset = 0.2)
    
    #text (hom[hom$V5<0.01,c('fold_change', '-logP')], hom$Motif[hom$V5<0.01], 
    #  pos=c(2,4), cex=0.9,offset = 0.5)
}
mtext(exp)
}
dev.off()

png 
  2

### Run HOMER

In [63]:
lf    = list.files("deseq_ATAC_NEW_bedfiles")
ups   = lf[grepl("up",lf) & !grepl("All",lf)]
downs = lf[grepl("down",lf) & !grepl("All",lf)]
bup   = lf[grepl("up",lf) & grepl("All",lf)]
bdown = lf[grepl("down",lf) & grepl("All",lf)]

In [74]:
beddir ="/nfs/lab/projects/islet_cytok/analysis/deseq_ATAC_NEW_bedfiles"

hdir = "/nfs/lab/projects/islet_cytok/analysis/HOMER_diff_sites/"

run_homer = function(bedfile, outdir,bgfile )
{
    system(paste ('findMotifsGenome.pl', bedfile, '/nfs/lab/KG/ref/male.masked.hg19.fa', 
       outdir,  '-bg', bgfile,  '-nomotif' ))
}

In [80]:
mclapply(1:length(ups), function (i) run_homer(paste(beddir,ups[i], sep="/"), 
                                               paste0(hdir,gsub(".bed", "", ups[i]) ), 
                                               paste(beddir,bup, sep="/")),mc.cores = 18)

[[1]]
[1] 0

[[2]]
[1] 0

[[3]]
[1] 0

[[4]]
[1] 0

[[5]]
[1] 0

[[6]]
[1] 0

[[7]]
[1] 0

In [81]:
mclapply(1:length(ups), 
         function (i) run_homer(paste(beddir,downs[i], sep="/"), paste0(hdir,gsub(".bed", "", downs[i]) ), 
                                paste(beddir,bdown, sep="/")),mc.cores = 18)
  

[[1]]
[1] 0

[[2]]
[1] 0

[[3]]
[1] 0

[[4]]
[1] 0

[[5]]
[1] 0

[[6]]
[1] 0

[[7]]
[1] 0

### Plot new motifs with  differential as bckground
up with up / down wth down

In [82]:
homer_dir = hdir
list.files(homer_dir)


[1] "cytokine_24hr_Hi.down"  "cytokine_24hr_Hi.up"    "cytokine_24hr_Hi2.down"
 [4] "cytokine_24hr_Hi2.up"   "cytokine_24hr_Lo.down"  "cytokine_24hr_Lo.up"   
 [7] "cytokine_24hr_Lo2.down" "cytokine_24hr_Lo2.up"   "cytokine_48hr_Hi.down" 
[10] "cytokine_48hr_Hi.up"    "cytokine_6hr_Hi.down"   "cytokine_6hr_Hi.up"    
[13] "cytokine_72hr_Hi.down"  "cytokine_72hr_Hi.up"

In [108]:
files2 = c('cytokine_6hr_Hi.up','cytokine_6hr_Hi.down',
           'cytokine_24hr_Hi2.up','cytokine_24hr_Hi2.down',
           'cytokine_24hr_Hi.up','cytokine_24hr_Hi.down',
            'cytokine_48hr_Hi.up','cytokine_48hr_Hi.down',
            'cytokine_72hr_Hi.up','cytokine_72hr_Hi.down')

In [112]:
pdf("cytokine_figures/Plot_motifs_diffbckg_log.pdf", height = 6, width=14)

par(mfcol=c(2,5), mar = c(4,4,4,0.2))

#par(mfcol=c(2,3), mar = c(4,4,4,0))
for (exp in files2){
hom= read.table( paste0(homer_dir, exp, '/knownResults.txt'), header=F, fill=T, check.names=F, skip=1, stringsAsFactors = F)
hom$fold_change = as.numeric(gsub('%',"", hom$V7)) /(as.numeric(gsub('%',"", hom$V9))+0.1) ##pseudocount for inf (only V2)
hom$Motif = toupper(str_split_fixed(hom$V1, "\\(",5)[,1])

    hom[,"-logP"] = -(as.numeric(hom$V4))
#hom[,"-logP"] = -log10(as.numeric(hom$V3))
 hom = hom[complete.cases(hom),]   
hom=hom[order(hom$fold_change, decreasing=T),]
hom=hom[order(hom[,'-logP'], decreasing=T),]
    
plot(hom[,c('fold_change', '-logP')], pch = 19, 
     col = (hom$V5<0.05)+1  )
if(sum(hom$V5<0.05)>0){
    text (hom[1:25,c('fold_change', '-logP')], hom$Motif[1:20], 
      pos=2, cex=0.7,offset = 0.2)
    
    #text (hom[hom$V5<0.01,c('fold_change', '-logP')], hom$Motif[hom$V5<0.01], 
    #  pos=c(2,4), cex=0.9,offset = 0.5)
}
mtext(exp)
}
dev.off()

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


png 
  2

### Plot the fold change by time

In [17]:
setwd("/nfs/lab/projects/islet_cytok/analysis/")

In [1]:
library(tidyr)

In [2]:
files3 = c('cytokine_6hr_Hi.up','cytokine_6hr_Hi.down',
           'cytokine_24hr_Hi.up','cytokine_24hr_Hi.down',
            'cytokine_48hr_Hi.up','cytokine_48hr_Hi.down',
            'cytokine_72hr_Hi.up','cytokine_72hr_Hi.down')

In [3]:
homer_dir = '/nfs/lab/elisha/islets/cytokine/HOMER/homer_dec15/'
#list.files(homer_dir)
files3 = c('hi_24_down','hi_24_up','hi2_24_down','hi2_24_up',
           'lo_24_down','lo_24_up','lo2_24_down','lo2_24_up',
           'hi_6_down','hi_6_up','hi_48_down','hi_48_up',
           'hi_72_down','hi_72_up')

In [7]:
df = data.frame()
for (exp in files3){
hom= read.table( paste0(homer_dir, exp, '/knownResults.txt'), header=F, fill=T, check.names=F, skip=1, stringsAsFactors = F)
hom$fold_change = as.numeric(gsub('%',"", hom$V7)) /(as.numeric(gsub('%',"", hom$V9))+0.1) ##pseudocount for inf (only V2)
hom$Motif = toupper(str_split_fixed(hom$V1, "\\(",5)[,1])

hom = hom[complete.cases(hom),] 
hom = subset(hom, Motif !="SEQBIAS:") 
hom$Qval = p.adjust(as.numeric(hom$V3))
tab = hom[,c("Motif","fold_change", "Qval")]
tab = tab[complete.cases(tab),]
tab$treat = exp
    df = rbind(df,tab )
}

In [8]:
head(df)

,Motif,fold_change,Qval,treat
,<chr>,<dbl>,<dbl>,<chr>
1,FOXA1,1.369912,3.64e-23,hi_24_down
2,FOXM1,1.381633,3.63e-21,hi_24_down
3,FOXA1,1.383023,3.63e-21,hi_24_down
4,FOX:EBOX,1.427441,3.61e-20,hi_24_down
5,FOXA2,1.406977,3.60e-17,hi_24_down
6,FOXA3,1.439490,3.59e-09,hi_24_down


In [9]:
df = df[!duplicated(df[,c(1,4)]),]
fc = spread(df[, c(1,2,4)], treat, fold_change)
qv = spread(df[, c(1,3,4)], treat, Qval)

rownames(fc) = fc$Motif
fc = fc[,-1]
rownames(qv) = qv$Motif
qv = qv[,-1]

fc = subset(fc, rowSums(qv<0.05, na.rm=T)>0)
qv = subset(qv, rowSums(qv<0.05, na.rm=T)>0)

In [10]:
fc = fc[order(rowSums(log10(qv))),]
qv = qv[order(rowSums(log10(qv))), ]

In [11]:
colnames(fc)

[1] "hi_24_down"  "hi_24_up"    "hi_48_down"  "hi_48_up"    "hi_6_down"  
 [6] "hi_6_up"     "hi_72_down"  "hi_72_up"    "hi2_24_down" "hi2_24_up"  
[11] "lo_24_down"  "lo_24_up"    "lo2_24_down" "lo2_24_up"

In [14]:
pdf("cytokine_figures/Motifs_by_time_all_bckg.pdf")
par(mfrow = c(5,5), mar =c(3,3,2,2), xpd = T)

#ups   = c(1,3,5,7)
#downs = c(2,4,6,8)

downs   = c(5,1,3,7)
ups = c(6,3,4,8)
for( i in 1:nrow(fc)){
    
    down = data.frame( time = c(6, 24, 48,  72),
                      fold_change = as.numeric(t(fc[i, downs])), qval = as.numeric(t(qv[i, c(1,3,5,7)])) )
    up = data.frame( time = c(6, 24, 48,  72),
                      fold_change = as.numeric(t(fc[i, ups])), qval = as.numeric(t(qv[i, c(2,4,6,8)])) )
    


plot(up[,1:2], type = "o", pch=19, col="orangered",
     axes=F,
     ylim = c(min(c(up[,2], down[,2])), max(c(up[,2], down[,2]))+ max(c(up[,2], down[,2]))/5   )  )
    axis(1, at=c(6, 24, 48,  72), labels = c(6, 24, 48,  72) )
    axis(2)
    mtext("Fold change", side=2 , line=2, cex=0.6)
    abline(h=1, col="gray", lty=2)
text(up[,c(1:2)], c("","*")[(up$qv<0.05) +1], pos=3, cex=2,offset = 0.1)
lines(down[,1:2], type = "o", pch=19, col="dodgerblue")
text(down[,c(1:2)], c("","*")[(down$qv<0.05) +1], pos=3, cex=2,offset = 0.1)
mtext( rownames(fc)[i], cex=0.8)
    }
dev.off()

png 
  2

In [15]:
ranks = apply(-fc, 2 , rank)


In [12]:
head(fc)

,hi_24_down,hi_24_up,hi_48_down,hi_48_up,hi_6_down,hi_6_up,hi_72_down,hi_72_up,hi2_24_down,hi2_24_up,lo_24_down,lo_24_up,lo2_24_down,lo2_24_up
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
IRF1,0.07142857,11.807692,0.15306122,6.946809,0.10126582,8.522222,0.12500000,6.843373,0.19148936,11.000000,0,8.166667,0,6.613095
IRF2,0.04705882,13.675000,0.08139535,7.894737,0.08571429,9.638889,0.09210526,7.924242,0.00000000,12.617647,0,14.957627,0,7.935714
IRF3,0.14893617,7.060000,0.26206897,4.979452,0.18421053,5.775510,0.25196850,4.819549,0.10370370,7.194444,0,8.567961,0,4.629167
IRF8,0.18840580,6.941748,0.26086957,5.074324,0.25217391,5.905405,0.24800000,4.940299,0.29104478,7.368056,0,5.764706,0,4.768240
ISRE,0.05000000,13.642857,0.05000000,7.814815,0.04000000,9.038462,0.07407407,7.500000,0.06896552,12.142857,0,12.923077,0,11.000000
PRDM1,0.48235294,3.833333,0.43103448,3.390000,0.59589041,3.877451,0.51592357,3.170213,0.59537572,4.368421,0,4.575875,0,4.099631


In [13]:
#downs   = c(5,11,13,1,9,3,7)
downs   = c(9,5,1,3,7)
downs   = c(5,1,3,7)
ups = downs+1

In [14]:
color = colorRampPalette(brewer.pal(9, "YlGnBu"))(50)

In [15]:
fcup   = fc[,ups]
qup    = qv[,ups]
fcup   = subset(fcup, rowSums(qup<0.05, na.rm=T)>0)
qup    = subset(qup, rowSums(qup<0.05, na.rm=T)>0)

xx = qup
xx[xx<0.05]<-"*"
xx[xx>=0.05]<-""

In [18]:
pheatmap(fcup, col=color, scale="row", cellwidth = 20,display_numbers = xx,
         number_color = "red",
         file = "cytokine_figures/Heatmap_motifs_up_only_hi.pdf",
         cluster_rows = T, cluster_cols = F)

In [22]:
pheatmap(log(fcup,2), col=color, scale="none", cellwidth = 20,display_numbers = xx,
         number_color = "red",
         file = "cytokine_figures/Heatmap_motifs_up_notscales.pdf",
         cluster_rows = T, cluster_cols = F)

In [19]:
fcdo   = fc[,downs]
qdo    = qv[,downs]
fcdo   = subset(fcdo, rowSums(qdo<0.05, na.rm=T)>0)
qdo    = subset(qdo, rowSums(qdo<0.05, na.rm=T)>0)

xx = qdo
xx[xx<0.05]<-"*"
xx[xx>=0.05]<-""

In [20]:
pheatmap(fcdo, col=color, scale="row", cellwidth = 20,display_numbers = xx,
         number_color = "red",
         file = "cytokine_figures/Heatmap_motifs_down_onlyhi.pdf",
         cluster_rows = T, cluster_cols = F)

In [31]:
l2 = log(fcdo,2)
l2[l2==-Inf]<-0

In [32]:
pheatmap(l2, col=color, scale="none", cellwidth = 20,display_numbers = xx,
         number_color = "red",
         file = "cytokine_figures/Heatmap_motifs_down_log2fc.pdf",
         cluster_rows = T, cluster_cols = F)

### Supplementary Table

In [21]:
homer_dir = '/nfs/lab/elisha/islets/cytokine/HOMER/homer_dec15/'
list.files(homer_dir)

[1] "bed_files"                "cytPeak_matrix.final.bed"
 [3] "hi_24_down"               "hi_24_up"                
 [5] "hi_48_down"               "hi_48_up"                
 [7] "hi_6_down"                "hi_6_up"                 
 [9] "hi_72_down"               "hi_72_up"                
[11] "hi2_24_down"              "hi2_24_up"               
[13] "hiVhi2_down"              "hiVhi2_up"               
[15] "homer.sh"                 "html"                    
[17] "lo_24_down"               "lo_24_up"                
[19] "lo2_24_down"              "lo2_24_up"               
[21] "output"                   "union_down"              
[23] "union_down.bed"           "union_down.txt"          
[25] "union_up"                 "union_up.bed"            
[27] "union_up.txt"             "union.sh"

In [22]:
files3 = c('union_down', 'union_up',
           'hi_6_down','hi_6_up',
           'hi_24_down','hi_24_up',
           'hi_48_down','hi_48_up',
           'hi_72_down','hi_72_up' ,
           'lo_24_down','lo_24_up',
           'lo2_24_down','lo2_24_up',
           'hi2_24_down','hi2_24_up')

In [66]:
cn = c('Motif','P_value', "log_P_value",'Q_value','No_seq_withMotif','Pct_seq_withMotif','Fold_change')

In [67]:
for (u in 0:1){
    for(s in seq(1,16,by=2)+u){
exp =     files3[s]
hom= read.table( paste0(homer_dir, exp, '/knownResults.txt'), header=F, fill=T, check.names=F, skip=1, stringsAsFactors = F)
hom$fold_change = as.numeric(gsub('%',"", hom$V7)) /(as.numeric(gsub('%',"", hom$V9))+0.1) ##pseudocount for inf (only V2)
hom$Motif2 = toupper(str_split_fixed(hom$V1, "\\(",5)[,1])
hom$Motif = toupper(str_split_fixed(hom$V1, "\\/",3)[,1])
hom = hom[complete.cases(hom),] 
hom = subset(hom, Motif2 !="SEQBIAS:") 
    
tab = hom[,c(12,3,4, 5,6,7,10)]
colnames(tab) = paste(cn, s , sep=".")
tab$Rank = 1:nrow(tab)
   if( s == (1+u) ) 
   {df =tab }else{
       df = merge(df, tab, by="Rank", all.x=T)
   }
}
    if(u==0){df_down=df}
      }

In [68]:
colnames(df) = colnames(df_down)=c("Rank", rep(cn, 8))

In [70]:
write.csv(df_down, "HOMER_diff_sites/SuppTab_down.csv", row.names=F )

In [71]:
write.csv(df, "HOMER_diff_sites/SuppTab_up.csv", row.names=F )

### GeoTable

In [11]:
mat = read.table('/nfs/lab/elisha/islets/cytokine/update_deseq/matrix/cytPeak.dec9.matrix.filtered.R', header=T, row.names=1)

In [15]:
meta = read.table('/nfs/lab/elisha/islets/cytokine/update_deseq/metadata/cytokine_35.metadata.txt', header=T)

In [18]:
meta$recode = meta$donor

In [19]:
meta$recode[meta$donor=="islet_1"]<-'SAMN12833535'
meta$recode[meta$donor=="islet_2"]<-'SAMN12889245'

In [21]:
table(meta$condition)


       hi       hi2        lo       lo2 untreated 
       11         5         3         3        13 

In [13]:
dim(mat)

[1] 165884     35

In [22]:
recode = c(hi2='2cyt-hi',
lo2='2cyt-lo',
hi='3cyt-hi',
lo='3cyt-lo',
untreated = 'untreat')

In [23]:
meta$treat = recode[meta$condition]

In [31]:
meta$sampname = paste(meta$recode, meta$treat, paste0(meta$time, "h"), sep="_")

In [39]:
colnames(mat) = meta$sampname[match( colnames(mat), meta$bam)]

In [42]:
rownames(mat) = gsub("peak-", "", rownames(mat))

In [43]:
write.table(mat, "../geo_submission_oct21/processed_files/Bulk_ATACSeq_count.matrix.txt", sep="\t", quote=F)

In [45]:
meta[meta$recode=='SAMN10977276',]

,donor,treatment,time,condition,bam_x,bam,bam_path,peaks,reads_1,reads_2,recode,treat,sampname
,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
11,SAMN10977276,SAMN10977276_hi,24,hi,X.nfs.lab.projects.islet_cytok.data.ATACseq.bam.SAMN10977276_hi.sort.filt.rmdup.bam,SAMN10977276_hi.sort.filt.rmdup.bam,/nfs/lab/projects/islet_cytok/data/ATACseq/bam/SAMN10977276_hi.sort.filt.rmdup.bam,/nfs/lab/projects/islet_cytok/data/ATACseq/peaks/SAMN10977276_hi_peaks.narrowPeak,/nfs/lab/mokino/islet_ATAC/27mar2019/SAMN10977276_hi_S1_L008_R1_001.fastq.gz,/nfs/lab/mokino/islet_ATAC/27mar2019/SAMN10977276_hi_S1_L008_R2_001.fastq.gz,SAMN10977276,3cyt-hi,SAMN10977276_3cyt-hi_24h
16,SAMN10977276,SAMN10977276_hi_noTNFA,24,hi2,X.nfs.lab.projects.islet_cytok.data.ATACseq.bam.hi_noTNFA_SAMN10977276.sort.filt.rmdup.bam,hi_noTNFA_SAMN10977276.sort.filt.rmdup.bam,/nfs/lab/projects/islet_cytok/data/ATACseq/bam/hi_noTNFA_SAMN10977276.sort.filt.rmdup.bam,/nfs/lab/projects/islet_cytok/data/ATACseq/peaks/hi_noTNFA_SAMN10977276_peaks.narrowPeak,/nfs/lab/mokino/islet_ATAC/27mar2019/SAMN10977276_hi_noTNFA_S2_L008_R1_001.fastq.gz,/nfs/lab/mokino/islet_ATAC/27mar2019/SAMN10977276_hi_noTNFA_S2_L008_R2_001.fastq.gz,SAMN10977276,2cyt-hi,SAMN10977276_2cyt-hi_24h
19,SAMN10977276,SAMN10977276_lo,24,lo,X.nfs.lab.projects.islet_cytok.data.ATACseq.bam.low_SAMN10977276.sort.filt.rmdup.bam,low_SAMN10977276.sort.filt.rmdup.bam,/nfs/lab/projects/islet_cytok/data/ATACseq/bam/low_SAMN10977276.sort.filt.rmdup.bam,/nfs/lab/projects/islet_cytok/data/ATACseq/peaks/low_SAMN10977276_peaks.narrowPeak,/nfs/lab/mokino/islet_ATAC/27mar2019/SAMN10977276_lo_S4_L008_R1_001.fastq.gz,/nfs/lab/mokino/islet_ATAC/27mar2019/SAMN10977276_lo_S4_L008_R2_001.fastq.gz,SAMN10977276,3cyt-lo,SAMN10977276_3cyt-lo_24h
22,SAMN10977276,SAMN10977276_lo_noTNFA,24,lo2,X.nfs.lab.projects.islet_cytok.data.ATACseq.bam.low_noTNFA_SAMN10977276.sort.filt.rmdup.bam,low_noTNFA_SAMN10977276.sort.filt.rmdup.bam,/nfs/lab/projects/islet_cytok/data/ATACseq/bam/low_noTNFA_SAMN10977276.sort.filt.rmdup.bam,/nfs/lab/projects/islet_cytok/data/ATACseq/peaks/low_noTNFA_SAMN10977276_peaks.narrowPeak,/nfs/lab/mokino/islet_ATAC/27mar2019/SAMN10977276_lo_noTNFA_S3_L008_R1_001.fastq.gz,/nfs/lab/mokino/islet_ATAC/27mar2019/SAMN10977276_lo_noTNFA_S3_L008_R2_001.fastq.gz,SAMN10977276,2cyt-lo,SAMN10977276_2cyt-lo_24h
35,SAMN10977276,SAMN10977276_untreated,24,untreated,X.nfs.lab.projects.islet_cytok.data.ATACseq.bam.untreated_SAMN10977276.sort.filt.rmdup.bam,untreated_SAMN10977276.sort.filt.rmdup.bam,/nfs/lab/projects/islet_cytok/data/ATACseq/bam/untreated_SAMN10977276.sort.filt.rmdup.bam,/nfs/lab/projects/islet_cytok/data/ATACseq/peaks/untreated_SAMN10977276_peaks.narrowPeak,/nfs/lab/mokino/islet_ATAC/27mar2019/SAMN10977276_untreated_S5_L008_R1_001.fastq.gz,/nfs/lab/mokino/islet_ATAC/27mar2019/SAMN10977276_untreated_S5_L008_R2_001.fastq.gz,SAMN10977276,untreat,SAMN10977276_untreat_24h
